In [1]:
from os import chdir
chdir('/home/jovyan')

In [2]:
from lib.postgres import connect_to_postgres

# Attribute Type
 
|    Column                                      |  Type                         |   
|-------------|---------------------- | 
age            | integer 
workclass      | text    
fnlwgt         | integer 
education      | text    
 education_num  | integer 
 marital_status | text    
 occupation     | text    
 relationship   | text    
 race           | text    
 gender         | text    
 capital_gain   | integer 
 capital_loss   | integer 
 hours_per_week | integer 
 native_country | text    
 income_label   | text    


In [3]:
con, cur = connect_to_postgres()
cur.execute("""
BEGIN;
ALTER TABLE adult ADD COLUMN _id SERIAL PRIMARY KEY;
ALTER TABLE adult ADD COLUMN target BOOLEAN;
COMMIT;
""")
con.close()

In [4]:
con, cur = connect_to_postgres()
cur.execute("""SELECT DISTINCT(income_label) FROM adult;""")
print(cur.fetchall())
con.close()

[(' <=50K',), (' >50K',)]


In [6]:
con, cur = connect_to_postgres()
cur.execute("""SELECT * FROM adult WHERE _id < 50;""")
print(cur.fetchall())
con.close()

[(39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K', 1, None), (50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K', 2, None), (38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K', 3, None), (53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K', 4, None), (28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K', 5, None), (37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K', 6, None), (49, ' Private', 16018

In [7]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label FROM adult WHERE target IS NULL;""")
this_id, income_label = cur.fetchone()

greater_than_50k = (income_label == ' >50K')
cur.execute("""
BEGIN;
UPDATE adult
SET target = {}
WHERE _id = {};
COMMIT;
""".format(greater_than_50k, this_id))

con.close()

In [8]:
con, cur = connect_to_postgres()
cur.execute("""
SELECT _id, income_label, target
FROM adult WHERE _id = {};
""".format(this_id))
print (this_id, cur.fetchone())
con.close()

1 (1, ' <=50K', False)


In [9]:
def encode_target(_id):
    """Encode the target for a single row as a boolean value. Takes a row _id."""
    con, cur = connect_to_postgres()
    cur.execute("""SELECT _id, income_label FROM adult WHERE _id = {}""".format(_id))
    this_id, income_label = cur.fetchone()
    assert this_id == _id
    
    greater_than_50k = (income_label == ' >50k')
    
    cur.execute("""
        BEGIN;
        UPDATE adult
        SET target = {}
        WHERE _id = {};
        COMMIT;
        """.format(greater_than_50k, _id))
    con.close()

In [11]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id FROM adult WHERE target IS NULL;""")
this_id, = cur.fetchone()
encode_target(this_id)
con.close()

In [12]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label, target FROM adult WHERE _id = {};""".format(this_id))
print(this_id, cur.fetchone())
con.close()

2 (2, ' <=50K', False)
